### The forward and backward passes

In [1]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from urllib.request import urlretrieve
from torch import tensor
from fastcore.test import test_close

torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

with gzip.open(path_gz, 'rb') as f: ((x_trn,y_trn),(x_val,y_val),_) = pickle.load(f, encoding='latin-1')
x_trn,y_trn, x_val,y_val = map(tensor, (x_trn,y_trn, x_val,y_val))

In [2]:
x_trn.shape, y_trn.shape, x_val.shape, y_val.shape

(torch.Size([50000, 784]),
 torch.Size([50000]),
 torch.Size([10000, 784]),
 torch.Size([10000]))

### Foundations version

#### Basic architecture

In [3]:
n,m = x_trn.shape
c = y_trn.max()+1
n,m,c

(50000, 784, tensor(10))

In [4]:
# num hidden
nh = 50

In [5]:
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)

In [6]:
def lin(x,w,b): return x@w+b

In [7]:
t = lin(x_val,w1,b1)
t.shape

torch.Size([10000, 50])

In [8]:
def relu(x): return x.clamp_min(0.)

In [9]:
t = relu(t)
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

In [10]:
def model(xb):
    l1 = lin(xb,w1,b1)
    l2 = relu(l1)
    return lin(l2,w2,b2)

In [11]:
res = model(x_val)
res.shape

torch.Size([10000, 1])

#### Loss function: MSE

(Of course `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [12]:
res.shape,y_val.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [13]:
(res-y_val).shape

torch.Size([10000, 10000])

We need to get rid of that trailing (,1) in order to use `mse`

In [14]:
res[:,0].shape

torch.Size([10000])

In [15]:
res.squeeze().shape

torch.Size([10000])

In [16]:
(res[:,0]-y_val).shape

torch.Size([10000])

In [17]:
y_trn.dtype

torch.int64

In [18]:
y_trn,y_val = y_trn.float(),y_val.float()

In [19]:
preds = model(x_trn)
preds.shape

torch.Size([50000, 1])

In [20]:
def mse(output, targ): return (output[:,0]-targ).pow(2).mean()

In [21]:
mse(preds, y_trn)

tensor(4308.76)

#### Gradient and backwards pass

In [22]:
from sympy import symbols,diff
x,y = symbols('x y')
diff(x**2, x)

2*x

In [23]:
diff(3*x**2+9,x)

6*x

In [24]:
def lin_grad(inp, out, w, b):
    # grad of matmul w.r.t. input
    # lin = x@w + b
    # -> d/dw lin = w
    # -> d/dw lin = x
    # -> d/db lin = 1
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum()

In [25]:
bla_in = tensor([[1.,0,1], [2,0,-2]])
bla_w = tensor([[10],[5],[1.5]])
bla_b  = tensor([100])

bla_in.shape, bla_w.shape, bla_b.shape

(torch.Size([2, 3]), torch.Size([3, 1]), torch.Size([1]))

In [26]:
bla_out = lin(bla_in, bla_w, bla_b)
bla_out.shape, bla_out

(torch.Size([2, 1]),
 tensor([[111.50],
         [117.00]]))

Note Umer: Let's set the gradien of bla_out manually & see how the gradients of bla_in, bla_w & bla_b arise

bla_out has dim 2x1 and usually would affect a scalar final outout (the loss). The gradient of the loss wrt bla_out is therefore 1x2 (1 value, 2 inputs), ie shape bla_out.T!

In [27]:
bla_out.g = tensor([[9.], [7]])

**Q: Shape of `out.g`** 

I would have expected `bla_out.g` to have shape 1x2 (e.g. `[[9, 7]]`), which is the shape of `bla_out.T`<br/>
Bu when doing `lin_grad` the shapes of `bla_out.g` (1x2) and `w.t()` (1x3) don't match.<br/>
Is the transpose I derived above already the one used with `w` in `lin_grad`?

In [28]:
lin_grad(bla_in, bla_out, bla_w, bla_b)

Okay, let's get back to it

In [29]:
def lin_grad(inp, out, w, b):
    # grad of matmul wrt input
    inp.g = out.g @ w.t()
    w.g = inp.t() @ out.g # (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

**TOOD: Understand & Derive**

`(inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)` is same as `inp.t() @ out.g`

In [30]:
def forward_and_backward(inp, targ):
    # forward pass
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0]-targ
    loss = diff.pow(2).mean()
    
    # backward pass
    out.g = 2.*diff[:,None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad(inp, l1, w1, b1)

In [31]:
forward_and_backward(x_trn, y_trn)

In [32]:
# Save for testing against later
def get_grad(x): return x.g.clone()
chks = w1,w2,b1,b2,x_trn
grads = w1g,w2g,b1g,b2g,ig = tuple(map(get_grad, chks))

We cheat a little bit and use PyTorch autograd to check our results.


In [33]:
def mkgrad(x): return x.clone().requires_grad_(True)
ptgrags = w1pt,w2pt,b1pt,b2pt,x_trn_pt = tuple(map(mkgrad, chks))

In [34]:
def forward(inp, targ):
    l1 = lin(inp, w1pt, b1pt)
    l2 = relu(l1)
    out = lin(l2, w2pt, b2pt)
    return mse(out, targ)

In [35]:
loss = forward(x_trn_pt, y_trn)
loss.backward()

In [36]:
for a,b in zip(grads, ptgrags): test_close(a,b.grad, eps=0.01)

### Refactor model

#### Layers as classes

In [37]:
class Relu():
    def __call__(self,inp):
        self.inp = inp
        self.out = inp.clamp_min(0)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [38]:
class Lin():
    def __init__(self, w, b): self.w,self.b=w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp,self.w,self.b)
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [39]:
class Mse():
    def __call__(self, inp, targ):
        self.inp,self.targ = inp,targ
        self.out = mse(inp, targ)
        return self.out
        
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [40]:
class Model():
    def __init__(self, w1,b1,w2,b2):
        self.layers = [Lin(w1,b1),Relu(),Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x=l(x)
        return self.loss(x,targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [41]:
model = Model(w1,b1,w2,b2)

In [42]:
loss = model(x_trn, y_trn)

In [43]:
model.backward()

In [44]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_trn.g, eps=0.01)

### Autograd

In [45]:
from torch import nn
import torch.nn.functional as F

In [46]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in, n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
        
    def forward(self, inp): return inp@self.w+self.b

In [47]:
Linear(10, 2)

Linear()

In [48]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh,n_out)]
    
    def forward(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:,None])

In [49]:
m  # pixels in input image

784

In [50]:
model = Model(m, nh, 1)

In [51]:
loss = model(x_trn, y_trn)

In [52]:
loss.backward()

In [53]:
l0 = model.layers[0]
l0.b.grad

tensor([    15.75,     27.66,     -6.92,     -5.88,     29.97,     41.93,    -17.70,     45.84,     24.61,    -67.37,
            -1.27,     26.90,      8.22,      2.10,      0.75,     18.63,     -0.00,      6.75,     46.79,     -2.14,
             9.23,     -2.20,     33.68,     24.67,    -32.96,      1.12,     11.09,     16.56,     -3.82,    -16.97,
             4.63,    -26.53,    -29.42,      0.37,     10.49,      6.41,     -6.55,    -12.32,    -47.34,     12.76,
            -1.23,     23.69,      3.46,    -14.35,     -2.32,    -11.71,      2.74,     -9.38,     25.01,     49.08])